## Recommending high value playlists for free user 

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from datetime import date
from datetime import time
import datetime as dt

#### Đọc dữ liệu

In [59]:
order_detail_df = pd.read_csv("data/order_processed.csv")
user_df = pd.read_csv("data/user_processed.csv")
listening_detail_df = pd.read_csv("data/listening_processed.csv")

In [42]:
order_detail_df.head(3)

,OrderID (PK),Transaction Date,UserID (FK),Bill Type,Total Amount
0,6706,2020-08-16,44822,VIP,199000
1,6998,2020-08-23,47128,VIP,199000
2,6927,2020-08-21,47134,VIP,199000


In [43]:
order_detail_df["Transaction Date"]=order_detail_df["Transaction Date"].astype("datetime64")

In [44]:
order_detail_df["Transaction Date"].describe()

<ipython-input-44-0c890eeed07c>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  order_detail_df["Transaction Date"].describe()


count                    3575
unique                    163
top       2020-12-13 00:00:00
freq                       60
first     2020-07-11 00:00:00
last      2020-12-31 00:00:00
Name: Transaction Date, dtype: object

In [45]:
order_summary_df = order_detail_df.groupby("UserID (FK)").agg(
                    {'Transaction Date': ['min', 'max', 'count']
                    }
                    ).reset_index()
order_summary_df.columns = ['User_ID','min_purchase_date','max_purchase_date','order_cnt']

order_summary_df.head(3)

,User_ID,min_purchase_date,max_purchase_date,order_cnt
0,40843,2020-09-04,2020-09-25,3
1,40861,2020-09-09,2020-09-09,1
2,41003,2020-10-27,2020-10-27,1


In [46]:
user_df.head(3)

,UserID (PK),VIP hay Free,Gender,Year of Birth,Age Range,Age,Region,Operation Systems,Registration Date,Total Listerning,Total Listening Time (Min)
0,86229,free,male,NaN,No information,NaN,VN,android,2020-11-27,218,4151.3
1,48053,free,male,1995.0,25 - 34,26.0,VN,ios,2020-08-24,19,278.6
2,46340,free,male,1988.0,25 - 34,33.0,VN,website,2020-08-19,28,170.5


In [47]:
user_df["Registration Date"].describe()

count          66146
unique           184
top       2020-10-31
freq            1219
Name: Registration Date, dtype: object

In [48]:
listening_detail_df.head(3)

,PlaylistID (PK),Playlist Name,Category,Sub Category,Playlist Type,Playlist Duration (min),Actual Duration (min),UserID (FK),Listening Time,Listening Date,Listening Datetime,Unnamed: 11
0,4529,Dạy Tiếng Anh Cho Trẻ Mầm Non,Podcast,Học ngoại ngữ,free,44.9,0.1,82846,06:05:00,03-12-2020,2020-12-03 06:05:00,NaN
1,4577,Tư Duy Tích Cực - Bạn Là Những Gì Bạn Nghĩ (Nh...,Sách nói,Hạnh phúc,vip,79.8,15.9,56112,12:49:00,30-10-2020,2020-10-30 12:49:00,NaN
2,4169,50 thiên thần của bạn: Giải phóng bản thân khỏ...,Sách nói,Tâm linh,vip,324.8,6.8,94362,00:17:00,15-12-2020,2020-12-15 00:17:00,NaN


In [49]:
listening_detail_df.dtypes

PlaylistID (PK)              int64
Playlist Name               object
Category                    object
Sub Category                object
Playlist Type               object
Playlist Duration (min)    float64
Actual Duration (min)      float64
UserID (FK)                  int64
Listening Time              object
Listening Date              object
Listening Datetime          object
Unnamed: 11                float64
dtype: object

In [60]:
# # Chuyển Ngày phát sinh lượt nghe sang kiểu dữ liệu date
# listening_detail_df["Listening Date"] = listening_detail_df["Listening Date"].map(
#                                         lambda x:date(int(x.split("-")[2]),int(x.split("-")[1]),int(x.split("-")[0])))
listening_detail_df["Listening Date"] = listening_detail_df["Listening Date"].astype("datetime64")

In [61]:
listening_detail_df["Listening Date"].describe()

<ipython-input-61-d0c3d75ad95d>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  listening_detail_df["Listening Date"].describe()


count                  750098
unique                    184
top       2020-12-27 00:00:00
freq                    10267
first     2020-01-07 00:00:00
last      2020-12-31 00:00:00
Name: Listening Date, dtype: object

### Lấy High Value Users

In [51]:
clustering_df = pd.read_csv("data/clustering_processed.csv")

In [7]:
clustering_df.columns

Index(['User_ID (FK)', 'Type_user', 'MainCluster_ID',
       'MainCluster_Description'],
      dtype='object')

In [52]:
clustering_df[clustering_df['Type_user'] == 'free'].groupby(['MainCluster_ID','MainCluster_Description']).size()

MainCluster_ID  MainCluster_Description        
1               Hibernating                        3093
2               Defrost                            1185
3               Need Attention                     6196
4               Lost active user                   2510
5               New User, Skimming Listeners       4256
6               New Users, Engaging Listeners      4099
7               Potential, Skimming Listeners      1090
8               Potential, Engaging Listeners      1622
9               Loyal Users, Skimming Listeners     342
10              Loyal Users, Engaging Listeners     427
dtype: int64

In [53]:
clustering_df[clustering_df['Type_user'] == 'paid'].groupby(['MainCluster_ID','MainCluster_Description']).size()

MainCluster_ID  MainCluster_Description
1               Churn App Users            387
2               Need attention             175
3               Lost Users                 254
4               Lost High-value            119
5               New Paid Users             424
6               Potential Users            727
7               Loyal Users                552
dtype: int64

In [54]:
## low_value user (for both free and paid users) have MainCluster_ID = 1/2/3
low_value_cluster_ids = [1,2]
clustering_df['Value Rank'] = clustering_df['MainCluster_ID'].map(lambda x: 'Low Value' if x in low_value_cluster_ids else 'High Value')

In [55]:
clustering_df.groupby(['Type_user','Value Rank','MainCluster_ID']).size()

Type_user  Value Rank  MainCluster_ID
free       High Value  3                 6196
                       4                 2510
                       5                 4256
                       6                 4099
                       7                 1090
                       8                 1622
                       9                  342
                       10                 427
           Low Value   1                 3093
                       2                 1185
paid       High Value  3                  254
                       4                  119
                       5                  424
                       6                  727
                       7                  552
           Low Value   1                  387
                       2                  175
dtype: int64

#### Nối vào listening_detail_df

In [37]:
# listening_detail_df_1 = listening_detail_df.copy()

In [63]:
listening_detail_df.columns

Index(['PlaylistID (PK)', 'Playlist Name', 'Category', 'Sub Category',
       'Playlist Type', 'Playlist Duration (min)', 'Actual Duration (min)',
       'UserID (FK)', 'Listening Time', 'Listening Date', 'Listening Datetime',
       'Unnamed: 11'],
      dtype='object')

In [64]:
is_high_value = clustering_df['Value Rank'] == 'High Value'
listening_detail_df=listening_detail_df.join(other=clustering_df[is_high_value][['User_ID (FK)','Type_user','Value Rank']].set_index('User_ID (FK)'),
                                             on = 'UserID (FK)', how = 'inner')

In [65]:
listening_detail_df.shape[0]

682545

	2. Use Cases:							
	a. Content Bundle: Gộp các contents liên quan để offer cho users theo bundle							
		1. Chọn những Content Bundle của những playlist có High Content Value ở (Output 1) >> Bundle High Content Value						
		2. Offer theo từng user						
			2.1 Application					
				2.1.1. Recommend trên trang chủ: Customize dựa trên lịch sử của người dùng, nếu là KH mới dẩy all top High Value content				
				2.2.1. Improve feature mới: Thường được nghe cùng >> Recommend trong phần dưới cùng của playlist khi user search và bấm vào playlist (~cat)				
			2.2 Streamlit					
				2.2.1. Input list các userid				
				2.2.2.	Output			
					- List các playlist recommend cho từng user (sorting)			

### Lấy highly converting value content

Những content có giá trị chuyển đổi cao là những content có thời lượng  trung bình 1 user nghe (trong vòng 2 tuần đó) trước khi chuyển đổi thành paid user  lớn hơn 20'

In [30]:
order_summary_df.head(3)

,User_ID,min_purchase_date,max_purchase_date,order_cnt
0,40843,2020-09-04,2020-09-25,3
1,40861,2020-09-09,2020-09-09,1
2,41003,2020-10-27,2020-10-27,1


In [31]:
listening_detail_df.columns

Index(['PlaylistID (PK)', 'Playlist Name', 'Category', 'Sub Category',
       'Playlist Type', 'Playlist Duration (min)', 'Actual Duration (min)',
       'UserID (FK)', 'Listening Time', 'Listening Date', 'Listening Datetime',
       'Unnamed: 11', 'Type_user', 'Value Rank'],
      dtype='object')

In [66]:
listen_pay_user_df = listening_detail_df.join(other=order_summary_df[["User_ID","min_purchase_date"]].set_index("User_ID"),
                                             how="inner",on="UserID (FK)")

In [67]:
listen_pay_user_df["botton_date"]=listen_pay_user_df["min_purchase_date"].map(lambda x: (x - dt.timedelta(days = 14)))
listen_pay_user_df["botton_date"]=listen_pay_user_df["botton_date"].astype("datetime64")
listen_pay_user_df.columns

Index(['PlaylistID (PK)', 'Playlist Name', 'Category', 'Sub Category',
       'Playlist Type', 'Playlist Duration (min)', 'Actual Duration (min)',
       'UserID (FK)', 'Listening Time', 'Listening Date', 'Listening Datetime',
       'Unnamed: 11', 'Type_user', 'Value Rank', 'min_purchase_date',
       'botton_date'],
      dtype='object')

##### Lấy những content được nghe trước ngày có first order trong vòng 2 tuần

In [68]:
hv_content_df = listen_pay_user_df[listen_pay_user_df["Listening Date"] > listen_pay_user_df["botton_date"]]
hv_content_df = hv_content_df [hv_content_df["Listening Date"]<hv_content_df["min_purchase_date"]]

In [69]:
hv_content_df= hv_content_df[['PlaylistID (PK)', 'Playlist Name', 'Category', 'Sub Category','Actual Duration (min)','UserID (FK)']]

##### Tìm những playlists có thời lượng  trung bình 1 user nghe (trong vòng 2 tuần đó) trước khi chuyển đổi thành paid user  lớn hơn 20'

In [70]:
hv_content_summary_df=hv_content_df.groupby(['PlaylistID (PK)','Playlist Name','Category', 'Sub Category','UserID (FK)']).agg(
    {'Actual Duration (min)': 'sum'}).reset_index()
hv_content_summary_df.columns = ['PlaylistID','PlaylistName','Category', 'SubCategory','UserID','TotalDuration']

In [71]:
hv_convert_content_df=hv_content_summary_df.groupby(["PlaylistID","PlaylistName",'Category', 'SubCategory']).agg({"TotalDuration":"mean"}).reset_index()
hv_convert_content_df.columns=["PlaylistID","PlaylistName",'Category', 'SubCategory',"AvgDuration"]
hv_convert_content_df=hv_convert_content_df.sort_values(by=["AvgDuration"],ascending = False)

In [72]:
hv_convert_content_df=hv_convert_content_df.sort_values(by=["AvgDuration"],ascending = False)

In [73]:
kq=hv_convert_content_df[hv_convert_content_df["AvgDuration"]>=20]
kq.shape[0]

414

In [74]:
hv_convert_content_df.describe(include="all")

,PlaylistID,PlaylistName,Category,SubCategory,AvgDuration
count,948.000000,948,948,948,948.000000
unique,NaN,948,5,31,NaN
top,NaN,10 Đức Tính Cần Có Của Người Xuất Gia,Sách nói,Thiếu nhi,NaN
freq,NaN,1,449,106,NaN
mean,4596.553797,NaN,NaN,NaN,23.422683
std,386.071764,NaN,NaN,NaN,24.673593
min,3808.000000,NaN,NaN,NaN,0.000000
25%,4305.750000,NaN,NaN,NaN,8.044643
50%,4602.500000,NaN,NaN,NaN,16.916667
75%,4917.250000,NaN,NaN,NaN,31.450526


In [75]:
kq.describe(include="all")

,PlaylistID,PlaylistName,Category,SubCategory,AvgDuration
count,414.000000,414,414,414,414.000000
unique,NaN,414,5,28,NaN
top,NaN,69,Sách nói,Thành công,NaN
freq,NaN,1,263,47,NaN
mean,4628.154589,NaN,NaN,NaN,41.831872
std,393.658362,NaN,NaN,NaN,27.354147
min,3808.000000,NaN,NaN,NaN,20.000000
25%,4291.500000,NaN,NaN,NaN,25.885417
50%,4703.500000,NaN,NaN,NaN,33.980000
75%,4945.250000,NaN,NaN,NaN,47.075000


In [76]:
kq.groupby("SubCategory").size()

SubCategory
Con cái                     16
Công nghệ                    3
Danh nhân                   11
Hạnh phúc                   31
Học ngoại ngữ                2
Kinh doanh                  29
Kinh dị                      3
Kinh điển Quốc tế           20
Kinh điển Việt Nam          20
Kiến thức                    7
Kỹ năng                     34
Lãnh đạo                    14
Lịch sử                      8
Lứa đôi                     13
Marketing                   11
Ngôn tình                   11
Pháp thoại                  15
Sách tóm tắt                26
Sức khoẻ                     5
Thiếu nhi                   13
Thành công                  47
Trinh thám                   9
Tâm linh                    38
Tâm lý                       3
Tâm sự                       2
Văn hoá                      2
Văn học Quốc tế Hiện đại    11
Văn học VN Hiện đại         10
dtype: int64

In [91]:
kq.to_csv("streamlit_data/hv_convert_content.csv",index=False)

### Basket_analysis - Chọn những nội dung hay nghe cùng nhau

In [92]:
## đọc kết quả basket analysis
basket_df=pd.read_excel("streamlit_data/basket_output.xlsx")

In [79]:
hv_convert_content_df.head(3)

,PlaylistID,PlaylistName,Category,SubCategory,AvgDuration
804,5036,Cạm Bẫy Trong Quản Lý,Sách nói,Lãnh đạo,237.533333
917,5195,Hỏa Ngục,Truyện nói,Trinh thám,220.712500
529,4679,Sức mạnh của ngôn từ,Sách nói,Kỹ năng,208.866667


## Content Bundle
 Cho UserID --> recommend content cho user


### 1. Recommend list based on historycal playlist and basket analysis

In [230]:
listening_detail_df.columns

Index(['PlaylistID (PK)', 'Playlist Name', 'Category', 'Sub Category',
       'Playlist Type', 'Playlist Duration (min)', 'Actual Duration (min)',
       'UserID (FK)', 'Listening Time', 'Listening Date', 'Listening Datetime',
       'Unnamed: 11', 'Type_user'],
      dtype='object')

In [210]:
listening_detail_df.shape[0]

750098

In [80]:
listen_free_user_df = listening_detail_df.join(other=order_summary_df[["User_ID","min_purchase_date"]].set_index("User_ID"),
                                             how="left",on="UserID (FK)")
listen_free_user_df = listen_free_user_df[listen_free_user_df["min_purchase_date"].isnull()]
listen_free_user_df.shape[0]

462582

In [232]:
listen_free_user_df.columns

Index(['PlaylistID (PK)', 'Playlist Name', 'Category', 'Sub Category',
       'Playlist Type', 'Playlist Duration (min)', 'Actual Duration (min)',
       'UserID (FK)', 'Listening Time', 'Listening Date', 'Listening Datetime',
       'Unnamed: 11', 'Type_user', 'min_purchase_date'],
      dtype='object')

### Recommendation Playlists

In [82]:
## Function suggest together_playlists base on basket analysis

def define_hardPlaylist(user_id, listen_free_user_df):
    df = listen_free_user_df[listen_free_user_df["UserID (FK)"]==user_id]
    bottom_date = df["Listening Date"].max() -dt.timedelta(days = 14)
    playlist = df[df["Listening Date"]>bottom_date]
    playlist_summary=playlist.groupby(["UserID (FK)","PlaylistID (PK)","Playlist Name"]).agg(
    {"Actual Duration (min)":["sum"]}).reset_index()
    playlist_summary.columns=["UserID","PlaylistID","PlaylistName","TotalDuration"]
    hard_playlist= playlist_summary.loc[playlist_summary["TotalDuration"].idxmax(),["PlaylistID","PlaylistName"]]
    return hard_playlist[1]

def suggest_together_playlists(user_id,listen_free_user_df):
    playlist = define_hardPlaylist(user_id, listen_free_user_df)
    together_playlists = basket_df[basket_df["antecedents"] == playlist]["next playlist"].to_list()
    if len(together_playlists)>1:
        return (together_playlists[0].split(","))
    else:
        return (together_playlists)
        
def extract_high_value(x,high_value_user):
#     high_value_user = listening_df[listening_df['Value Rank'] == 'High Value']
    high_value_user_filter = high_value_user[high_value_user['UserID (FK)'] == x]
    last_listening = high_value_user_filter.groupby(by='UserID (FK)')['Listening Date'].max().reset_index()
    last_listening.columns = ['UserID (FK)','Last Listening Date']
    test = pd.merge(high_value_user_filter, last_listening, how='left', on= 'UserID (FK)')
    test['Last Listening Date'] = test['Last Listening Date'].astype('datetime64')
    test['time window'] = (test['Last Listening Date'] - test['Listening Date']).dt.days
    test = test[test['time window'] <= 14]
    test = test.groupby(by=['UserID (FK)', 'Sub Category'])['PlaylistID (PK)'].count().reset_index()
    test.columns = ['UserID (FK)', 'Sub Category', 'Listening Count']
    test = test[test['Listening Count'] == test['Listening Count'].max()]
    test = pd.merge(test, hv_convert_content_df[['PlaylistID','PlaylistName', 'Category', 'SubCategory' , 'AvgDuration']], how='left', left_on='Sub Category', right_on = 'SubCategory' )
    lst = test['PlaylistName'].tolist()
    
    return lst

def recommend_playlists(user_id, listen_free_user_df):
    lst1 = extract_high_value(user_id,listen_free_user_df)
    lst2 = suggest_together_playlists(user_id,listen_free_user_df)
    lst3 = lst1 + lst2
    newlist=[ii for n,ii in enumerate(lst3) if ii not in lst3[:n]]
    
    return ((newlist[:3]+newlist[-3:]) if len(newlist)>6 else newlist )

In [123]:
# listen_free_user_df.to_csv("data/export/listen_free_user_df.csv")
# listening_detail_df.to_csv("data/export/listening_detail_df.csv")

In [ ]:
##### Thử chạy cho 10 free users

In [84]:
users_lst = listen_free_user_df['UserID (FK)'].unique().tolist()[:10]
recomemend_lst = {}
for user in users_lst:
    recomemend_lst[user] = recommend_playlists(user,listen_free_user_df) 
recomemend_df = pd.DataFrame()
recomemend_df["User_ID"]=recomemend_lst.keys()
recomemend_df["RecommendPlaylists"]=recomemend_lst.values()
recomemend_df

,User_ID,RecommendPlaylists
0,82846,"(Tin Vào Chính Mình (Eng), Đúng Việc, Nghĩ Lớn..."
1,56112,"(Sức mạnh của ngôn từ, Tại sao cần đơn giản?, ..."
2,94362,"(Muôn Kiếp Nhân Sinh - Tập 1, Tháo Dây Oan Trá..."
3,85469,"([Sách Tóm Tắt] Chú Chó Nhìn Thấy Gì, [Sách Tó..."
4,87429,"(Nhạc Tập Trung Làm Việc, Học Tập, Nhạc Thư Gi..."
5,64413,"(Dê Con Nhanh Trí, Ông già và biển cả, Umbalen..."
6,69697,"(Điểm Tin Trong Nước, Độc Đáo TV - Tin tức hàn..."
7,78301,"(Tin Vào Chính Mình (Eng), Đúng Việc, Nghĩ Lớn..."
8,64148,"([Sách Tóm Tắt] Chú Chó Nhìn Thấy Gì, [Sách Tó..."
9,42445,"(Tiệm Bánh Mì Podcast, Tây Sơn Ký: Những câu c..."


#### Lấy clustering name cho free user kèm theo clustering

In [85]:
clustering_df.columns

Index(['User_ID (FK)', 'Type_user', 'MainCluster_ID',
       'MainCluster_Description', 'Value Rank'],
      dtype='object')

In [86]:
free_clustering_df = clustering_df[clustering_df["Type_user"]=="free"]

In [87]:
listen_free_user_df ['Value Rank'].unique()

array(['High Value'], dtype=object)

#### Ghi vào file

In [88]:
listen_free_user_df = listen_free_user_df.join(other=free_clustering_df[["User_ID (FK)",'MainCluster_ID','MainCluster_Description']].set_index("User_ID (FK)"),
                        how = "inner", on = "UserID (FK)")
listen_free_user_df.to_csv("streamlit_data/listen_free_user_df.csv",index= False)

##### xem thử các clusters trong free users như thế nào

In [89]:
cluster = listen_free_user_df.groupby(["MainCluster_ID","MainCluster_Description"]).agg({"UserID (FK)":"count"}).reset_index()

cluster.columns=["MainCluster_ID","MainCluster_Description","#"]

In [90]:
cluster.sort_values(by=["MainCluster_ID"])

,MainCluster_ID,MainCluster_Description,#
0,3,Need Attention,67857
1,4,Lost active user,51783
2,5,"New User, Skimming Listeners",35089
3,6,"New Users, Engaging Listeners",145770
4,7,"Potential, Skimming Listeners",19558
5,8,"Potential, Engaging Listeners",98687
6,9,"Loyal Users, Skimming Listeners",17705
7,10,"Loyal Users, Engaging Listeners",26133
